##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 第4天 - 微调自定义模型

欢迎回到 Kaggle 5天生成式AI课程！

在本 notebook 中,你将使用 Gemini API 来微调一个自定义的、针对特定任务的模型。微调可以用于各种任务,从传统的 NLP 问题(如实体提取或总结)到创意性任务(如风格化生成)。你将微调一个模型,将一段文本(新闻组帖子)分类到它所属的类别(新闻组名称)。

本代码教程将指导你使用 API 进行模型微调。[AI Studio](https://aistudio.google.com/app/tune) 还支持直接在网页界面创建新的微调模型,让你能够使用来自 Google Sheets、Drive 或自己的文件中的数据快速创建和监控模型。

In [ ]:
%pip install -U -q 'google-generativeai>=0.8.3'

In [ ]:
import google.generativeai as genai

### Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [ ]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

If you received an error response along the lines of `No user secrets exist for kernel id ...`, then you need to add your API key via `Add-ons`, `Secrets` **and** enable it.

![Screenshot of the checkbox to enable GOOGLE_API_KEY secret](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_3.png)

### 探索可用模型

你将使用 [`TunedModel.create`](https://ai.google.dev/api/tuning#method:-tunedmodels.create) API 方法来启动微调任务并创建自定义模型。通过 [`models.list`](https://ai.google.dev/api/models#method:-models.list) 接口找到支持它的模型。你也可以在 [model tuning 文档](https://ai.google.dev/gemini-api/docs/model-tuning/tutorial?lang=python) 中找到更多关于模型微调的信息。

In [ ]:
for model in genai.list_models():
    if "createTunedModel" in model.supported_generation_methods:
        print(model.name)

## 下载数据集

在本实践中,你将使用与在 Keras 中训练分类器时相同的新闻组数据集。在这个例子中,你将使用一个经过微调的 Gemini 模型来实现相同的目标。

[20 Newsgroups Text Dataset](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) 包含18,000个新闻组帖子,涵盖20个主题,分为训练集和测试集。

In [ ]:
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset="train")
newsgroups_test = fetch_20newsgroups(subset="test")

# View list of class names for dataset
newsgroups_train.target_names

Here's what a single row looks like.

In [ ]:
print(newsgroups_train.data[0])

## 准备数据集

你将使用与第2天自定义模型相同的预处理代码。这个预处理会移除个人信息(这些信息可能被用来"快速定位"论坛的已知用户),并将文本格式化为更像普通文本而不是新闻组帖子的形式(例如,通过移除邮件头)。这种标准化允许模型泛化到普通文本,而不会过度依赖特定字段。如果你的输入数据始终是新闻组帖子,且这些结构提供了有效的信号,那么保留这些结构可能会有帮助。

In [ ]:
import email
import re
import pandas as pd


def preprocess_newsgroup_row(data):
    """
    预处理单条新闻组数据
    Args:
        data: 原始新闻组文本数据
    Returns:
        text: 处理后的文本
    """
    # 从原始文本解析出邮件格式的消息
    msg = email.message_from_string(data)
    # 只保留主题和正文内容,用两个换行符连接
    text = f"{msg['Subject']}\n\n{msg.get_payload()}"
    # 使用正则表达式移除所有电子邮件地址
    # [\w\.-]+ 匹配一个或多个字母数字字符、点或连字符
    # @ 匹配 @ 符号
    text = re.sub(r"[\w\.-]+@[\w\.-]+", "", text)
    # 将文本截断到40000个字符以适应模型输入限制
    text = text[:40000]

    return text


def preprocess_newsgroup_data(newsgroup_dataset):
    """
    预处理整个新闻组数据集
    Args:
        newsgroup_dataset: 原始新闻组数据集对象
    Returns:
        df: 处理后的 DataFrame
    """
    # 创建包含文本内容和标签的 DataFrame
    df = pd.DataFrame(
        {"Text": newsgroup_dataset.data, "Label": newsgroup_dataset.target}
    )
    # 对每条文本应用预处理函数
    df["Text"] = df["Text"].apply(preprocess_newsgroup_row)
    # 将数字标签映射为对应的类别名称
    df["Class Name"] = df["Label"].map(lambda l: newsgroup_dataset.target_names[l])

    return df

In [ ]:
# 对训练集和测试集应用预处理
# 使用之前定义的 preprocess_newsgroup_data 函数处理原始新闻组数据
# 处理训练数据集
df_train = preprocess_newsgroup_data(newsgroups_train)
# 处理测试数据集
df_test = preprocess_newsgroup_data(newsgroups_test)

# 显示训练数据集的前几行用于检查
df_train.head()

现在对数据进行采样。你将为每个类别保留50行用于训练。注意这比 Keras 示例中使用的还要少,因为这种技术(参数高效微调,即 PEFT)只更新相对少量的参数,不需要训练新模型或更新大模型。

In [ ]:
def sample_data(df, num_samples, classes_to_keep):
    """
    从数据集中采样数据
    Args:
        df: 输入的 DataFrame
        num_samples: 每个标签要采样的行数
        classes_to_keep: 要保留的类别的正则表达式模式
    Returns:
        采样后的 DataFrame
    """
    # 按 Label 列分组,对每个组采样指定数量的行
    # groupby("Label") 按标签分组
    # apply(lambda x: x.sample(num_samples)) 对每个组随机采样 num_samples 行
    df = (
        df.groupby("Label")[df.columns]
        .apply(lambda x: x.sample(num_samples))
        .reset_index(drop=True)  # 重置索引
    )

    # 只保留类名匹配指定模式的行
    # str.contains() 检查 Class Name 是否匹配正则表达式
    df = df[df["Class Name"].str.contains(classes_to_keep)]
    # 将 Class Name 列转换为 category 类型以节省内存
    df["Class Name"] = df["Class Name"].astype("category")

    return df


# 设置采样参数
TRAIN_NUM_SAMPLES = 50  # 训练集每个类别采样 50 行
TEST_NUM_SAMPLES = 10   # 测试集每个类别采样 10 行
# 只保留以 rec 和 sci 开头的类别
CLASSES_TO_KEEP = "^rec|^sci"

# 对训练集和测试集进行采样
df_train = sample_data(df_train, TRAIN_NUM_SAMPLES, CLASSES_TO_KEEP)
df_test = sample_data(df_test, TEST_NUM_SAMPLES, CLASSES_TO_KEEP)

## 评估基准性能

在开始模型微调之前,最佳实践是对可用模型进行评估,以确保你能够衡量微调带来的提升。

首先确定一个样本行用于直观检查。

In [ ]:
# 选择测试集中的第一条数据作为样本
sample_idx = 0

# 使用预处理函数处理测试集中的第一条新闻组数据
sample_row = preprocess_newsgroup_row(newsgroups_test.data[sample_idx])

# 获取这条数据的真实标签
sample_label = newsgroups_test.target_names[newsgroups_test.target[sample_idx]]

# 打印预处理后的文本内容和对应的标签
print(sample_row)
print('---')
print('Label:', sample_label)

直接将文本作为提示输入并不能得到期望的结果。模型会尝试对消息作出回应。

In [ ]:
baseline_model = genai.GenerativeModel("gemini-1.5-flash-001")
response = baseline_model.generate_content(sample_row)
print(response.text)

你可以使用本周学到的提示工程技术来引导模型执行所需的任务。尝试一些你自己的想法,看看哪些是有效的,或者查看接下来几个单元格中的不同方法。注意这些方法的有效性是不同的！

In [ ]:
# 使用零样本提示(zero-shot prompt)直接询问模型
# 零样本提示意味着不提供任何示例,直接让模型完成任务

# 定义提示语句,询问消息来自哪个新闻组
prompt = "From what newsgroup does the following message originate?"

# 调用模型进行推理
# [prompt, sample_row] 将提示和样本文本组合成列表传入
# generate_content() 生成回答内容
baseline_response = baseline_model.generate_content([prompt, sample_row])

# 打印模型的回答
print(baseline_response.text)

这种技术产生了相当冗长的响应。你可以尝试提取相关文本,或者进一步优化提示词。

In [ ]:
from google.api_core import retry

# 使用系统指令来进行更直接的提示,获得更简洁的答案
system_instruct = """
You are a classification service. You will be passed input that represents
a newsgroup post and you must respond with the newsgroup from which the post
originates.
"""

# 创建一个带有系统指令的生成式模型实例
instructed_model = genai.GenerativeModel("gemini-1.5-flash-001",
                                       system_instruction=system_instruct)

# 定义重试策略,在出现临时错误时进行重试
retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

# 定义预测标签的函数
def predict_label(post: str) -> str:
    """
    预测新闻组帖子所属的类别
    Args:
        post: 新闻组帖子文本
    Returns:
        预测的类别标签
    """
    # 使用模型生成预测结果,应用重试策略
    response = instructed_model.generate_content(sample_row, request_options=retry_policy)
    # 清理响应文本(去除首尾空白)并返回
    return response.text.strip()

# 对样本进行预测
prediction = predict_label(sample_row)

# 打印预测结果
print(prediction)
print()
# 检查预测是否正确
print("Correct!" if prediction == sample_label else "Incorrect.")

现在使用上面定义的函数运行一个简短的评估。测试集被进一步采样以确保实验在 API 的免费层上顺利运行。在实践中,你会对整个数据集进行评估。

In [ ]:
# 导入进度条库
from tqdm.rich import tqdm

# 为 pandas 操作启用进度条显示功能
tqdm.pandas()

# 从测试数据集中进一步采样,以符合免费配额限制
# 使用 sample_data 函数从每个类别采样 2 条数据
# '.*' 表示匹配所有类别
df_baseline_eval = sample_data(df_test, 2, '.*')

# 使用进度条对采样数据进行预测
# progress_apply 会显示预测进度
# predict_label 函数对每条文本进行分类预测
df_baseline_eval['Prediction'] = df_baseline_eval['Text'].progress_apply(predict_label)

# 计算预测准确率
# 比较 Class Name 和 Prediction 列是否相等
# sum() 计算相等的数量
# len(df_baseline_eval) 得到总样本数
accuracy = (df_baseline_eval["Class Name"] == df_baseline_eval["Prediction"]).sum() / len(df_baseline_eval)
# 以百分比形式打印准确率
print(f"Accuracy: {accuracy:.2%}")

现在查看数据框,比较预测结果和标签。

In [ ]:
df_baseline_eval

## 微调自定义模型

在这个例子中,你将使用微调来帮助创建一个模型,该模型不需要提示或系统指令,可以直接从训练数据中提供的类别输出简洁的文本。

数据包含输入文本(处理过的帖子)和输出文本(类别或新闻组),你可以用它们来开始微调模型。

用于微调的 Python SDK 支持 Pandas 数据框作为输入,所以你不需要任何自定义的数据生成器或管道。只需要指定输入和相关列作为 `input_key` 和 `output_key`。

在调用 `create_tuned_model` 时,你也可以指定模型微调的超参数：
 - `epoch_count`: 定义遍历数据的次数
 - `batch_size`: 定义在单个步骤中处理的行数
 - `learning_rate`: 定义每一步更新模型权重的缩放因子

你也可以选择省略这些参数而使用默认值。[了解更多](https://developers.google.com/machine-learning/crash-course/linear-regression/hyperparameters)关于这些参数及其工作原理。对于这个例子,这些参数是通过运行一些微调任务并选择既有效又快速的参数确定的。

In [ ]:
from collections.abc import Iterable
import random

# 生成模型 ID
# 添加随机数以增加模型 ID 的唯一性
# random.randint(10000, 99999) 生成 5 位随机数
model_id = f"newsgroup-classifier-{random.randint(10000, 99999)}"

# 创建并启动模型微调任务
tuning_op = genai.create_tuned_model(
    # 指定基础模型
    "models/gemini-1.5-flash-001-tuning",
    # 传入训练数据集
    training_data=df_train,
    # 指定输入列名(文本内容)
    input_key="Text",  
    # 指定输出列名(类别标签)
    output_key="Class Name",  
    # 设置模型 ID
    id=model_id,
    # 设置模型显示名称
    display_name="Newsgroup classification model",
    # 设置批次大小
    batch_size=16,
    # 设置训练轮数
    epoch_count=2,
)

# 打印生成的模型 ID,用于后续追踪和使用
print(model_id)

这创建了一个将在后台运行的微调任务。要检查微调任务的进度,运行这个单元格来绘制当前状态和损失曲线。一旦状态达到 `ACTIVE`,微调就完成了,模型就可以使用了。

微调任务会进入队列,所以最初可能看起来没有执行任何训练步骤,但它会逐步推进。微调可能需要20分钟以上,具体取决于数据集大小和微调基础设施的繁忙程度等因素。在等待时,不妨享用一杯好茶,或者到[Discord](https://discord.com/invite/kaggle)群组中跟[我本人](https://discord.com/users/132124213132787712)打个招呼。

随时可以安全地停止这个单元格。这不会停止微调任务。

In [ ]:
import time
import seaborn as sns

# 循环检查模型状态直到训练完成
# := 海象运算符用于赋值并返回值
# 当模型状态不是 'ACTIVE' 时继续循环
while (tuned_model := genai.get_tuned_model(f"tunedModels/{model_id}")).state.name != 'ACTIVE':

    # 如果有训练快照数据,绘制损失曲线
    if tuned_model.tuning_task.snapshots:
        # 将快照数据转换为 DataFrame
        snapshots = pd.DataFrame(tuned_model.tuning_task.snapshots)
        # 使用 seaborn 绘制训练损失随步数变化的曲线
        # x 轴是训练步数,y 轴是平均损失
        sns.lineplot(data=snapshots, x="step", y="mean_loss")
    
    # 打印当前模型状态
    print(tuned_model.state)

    # 等待 60 秒后再次检查
    time.sleep(60)

# 训练完成后打印最终状态
print(f"Done! The model is {tuned_model.state.name}")

## 使用新模型

现在你有了一个微调后的模型,试试用自定义数据来测试它。你使用与普通 Gemini API 交互相同的 API,但需要使用 `tunedModels/` 前缀来指定你的新模型作为模型名称。

In [ ]:
# 使用微调后的模型 ID 创建生成式模型实例
your_model = genai.GenerativeModel(f"tunedModels/{model_id}")

# 定义一个测试文本
# 这是一个关于太空旅行的查询文本
new_text = """
First-timer looking to get out of here.

Hi, I'm writing about my interest in travelling to the outer limits!

What kind of craft can I buy? What is easiest to access from this 3rd rock?

Let me know how to do that please.
"""

# 使用微调后的模型对新文本进行分类预测
response = your_model.generate_content(new_text)
# 打印模型的预测结果
print(response.text)

### 评估

你可以看到模型输出的标签与训练数据中的标签相对应,而且不需要任何系统指令或提示,这已经是一个很大的改进。现在看看它在测试集上的表现如何。

注意这个例子中没有并行处理;对测试子集进行分类将需要几分钟时间。

In [ ]:
def classify_text(text: str) -> str:
    """将提供的文本分类到已知的新闻组类别中"""
    # 使用微调后的模型对文本进行预测,应用重试策略
    response = your_model.generate_content(text, request_options=retry_policy)
    # 获取第一个候选结果
    rc = response.candidates[0]

    # 检查生成是否正常完成
    # 如果不是正常停止(可能是错误、过滤或重复等),返回错误标记
    if rc.finish_reason.name != "STOP":
        return "(error)"
    else:
        # 返回生成的文本内容(类别标签)
        return rc.content.parts[0].text


# 从测试集采样以减少配额使用
# 如果配额充足,建议使用完整测试集: df_model_eval = df_test.copy()
df_model_eval = sample_data(df_test, 4, '.*')  # 每类采样4条数据

# 对采样数据进行预测
# progress_apply 显示预测进度
# classify_text 函数对每条文本进行分类
df_model_eval["Prediction"] = df_model_eval["Text"].progress_apply(classify_text)

# 计算预测准确率
# 比较预测结果与真实标签是否相同
accuracy = (df_model_eval["Class Name"] == df_model_eval["Prediction"]).sum() / len(df_model_eval)
# 打印准确率(百分比格式)
print(f"Accuracy: {accuracy:.2%}")

## 比较令牌使用量

AI Studio 和 Gemini API 提供免费的模型微调,但是使用微调后的模型时仍需遵循正常的限制和收费标准。

输入提示的大小和其他生成配置(如系统指令),以及生成的输出令牌数量,都会影响请求的总体成本。

In [ ]:
# 计算带系统指令的基线模型的输入令牌数
sysint_tokens = instructed_model.count_tokens(sample_row).total_tokens
print(f'System instructed baseline model: {sysint_tokens} (input)')

# 计算微调模型的输入令牌数
tuned_tokens = your_model.count_tokens(sample_row).total_tokens
print(f'Tuned model: {tuned_tokens} (input)')

# 计算令牌节省比例
savings = (sysint_tokens - tuned_tokens) / tuned_tokens
print(f'Token savings: {savings:.2%}')  # Note that this is only n=1.

较早的冗长模型也产生了比此任务所需更多的输出令牌。

In [ ]:
# 获取基线模型的输出令牌数
# usage_metadata.candidates_token_count 获取生成内容的令牌数量
baseline_token_output = baseline_response.usage_metadata.candidates_token_count
# 打印基线模型(详细输出)的令牌数
print('Baseline (verbose) output tokens:', baseline_token_output)

# 使用微调模型对同样的样本进行预测
tuned_model_output = your_model.generate_content(sample_row)
# 获取微调模型的输出令牌数
tuned_tokens_output = tuned_model_output.usage_metadata.candidates_token_count
# 打印微调模型的输出令牌数
print('Tuned output tokens:', tuned_tokens_output)

## 后续步骤

现在你已经微调了一个分类模型,可以尝试其他任务,比如使用手写示例(甚至是生成的示例!)来微调模型,使其以特定的语气或风格回应。Kaggle 上有[许多数据集](https://www.kaggle.com/datasets)供你尝试。

了解[监督式微调最有效的场景](https://cloud.google.com/blog/products/ai-machine-learning/supervised-fine-tuning-for-gemini-llm)。

查看[微调教程](https://ai.google.dev/gemini-api/docs/model-tuning/tutorial?hl=en&lang=python),了解另一个示例,展示了微调模型如何扩展到训练数据之外的新的、未见过的输入。